# Visualize patched sphere simulations

In [2]:
%pwd

'/home/dkannan/git-remotes/protein_mobility'

In [3]:
import nglutils as ngu
import nglview as nv

import os
import sys
from pathlib import Path
import numpy as np
import scipy as sp

import mdtraj
import polychrom
from polychrom.hdf5_format import list_URIs, load_URI, load_hdf5_file

In [4]:
def extract_trajectory(simdir, wrap=False, start=0, end=-1, every_other=10):
    """Load conformations from a simulation trajectory stored in the hdf5 files in simdir.
    
    Parameters
    ----------
    simdir : str or Path
        path to simulation directory containing .h5 files
    start : int
        which time block to start loading conformations from
    end : int
        which time block to stop loading conformations from
    every_other : int
        skip every_other time steps when loading conformations
        
    Returns
    -------
    X : array_like (num_t, N, 3)
        x, y, z positions of all monomers over time
    
    """
    X = []
    data = list_URIs(simdir)
    #check if PBCbox was used
    initArgs = load_hdf5_file(Path(simdir)/"initArgs_0.h5")
    PBCbox = np.array(initArgs['PBCbox'])
    if PBCbox.any():
        boxsize = PBCbox
    if start == 0:
        starting_pos = load_hdf5_file(Path(simdir)/"starting_conformation_0.h5")['pos']
        X.append(starting_pos)
    for conformation in data[start:end:every_other]:
        pos = load_URI(conformation)['pos']
        if PBCbox.any() and wrap:
            mults = np.floor(pos / boxsize[None, :])
            pos = pos - mults * boxsize[None, :]
            assert pos.min() >= 0
        X.append(pos)
    X = np.array(X)
    return X

def patched_particle_geom(f, R=1):
    """ Distribute f residues on a sphere with equal angles."""
    
    #first position is center particle
    positions = [[0., 0., 0.]]
    theta = np.pi/2
    for i in range(min(f, 5)):
        #for valency less than 5, just distribute points on a circle in the x-y plane
        phi = 2 * np.pi * i / f
        x = R * np.sin(theta) * np.cos(phi)
        y = R * np.sin(theta) * np.sin(phi)
        z = R * np.cos(theta)
        positions.append([x, y, z])
    
    if f >=5:
        #octahedron -> put 5th particle perpendicular to plain of 4 points
        print("Have not implemented yet")
    return np.array(positions)

#make a topology for this f+1-atom chain
def mdtop_for_patched_particles(N, nchains, f, atom_names="XXX"):
    """
    Generate an mdtraj.Topology object for a single polymer of length N.

    Parameters
    ----------
    N : int
        total number of particles
    nchains : int
        number of tetramers
    f : int
        valency, i.e. number of patches
    atom_names : string (up to 3 characters) or list of such
        names for the atoms. This can be useful for labelling different types
        of monomers, such as compartments. If given as list, should have length
        N.

    Notes
    -----
        - use iron as element to prevent NGLView from calculating bonds
    """

    # Check that atom names are good
    if isinstance(atom_names, str):
        atom_names = [atom_names[:3] for _ in range(N)]
    elif not isinstance(atom_names, list) or not len(atom_names) == N:
        print("WARNING: atom_names should be a list of length N = {}. Using default." % N)
        atom_names = ["XXX" for _ in range(N)]

    # Generate topology
    top = mdtraj.Topology()
    resi = 0
    for i in range(nchains):
        ch = top.add_chain()
        res = top.add_residue("{0:03d}".format(resi // 10000), ch)
        first_atom = top.add_atom(atom_names[resi], mdtraj.element.iron, res)
        resi += 1
        for k in range(f):
            res = top.add_residue("{0:03d}".format(resi // 10000), ch)
            cur_atom = top.add_atom(atom_names[resi], mdtraj.element.iron, res)
            top.add_bond(first_atom, cur_atom)
            resi += 1
    return top


In [5]:
DATADIR = Path('/net/levsha/share/deepti/simulations/protein_mobility')

In [37]:
def make_animation(f, E0, N=1000, vol_fraction=0.3):
    Y = extract_trajectory(DATADIR/f"N{N}_f{f}_E0{E0}_v{vol_fraction}", wrap=False, 
                           start=0, end=1000, every_other=1)
    print(Y.shape)
    X = Y[-1]
    print(X.shape)
    N = int(Y.shape[1]/(f+1))
    print(N)
    print(X.shape)
    atom_names = (['B'] + ['A']*f) * N
    top = mdtop_for_patched_particles((f+1)*N, N, f, atom_names=atom_names)
    view = ngu.xyz2nglview(X, top=top)
    view.add_representation('ball+stick', selection='.A',
                                                colorScheme='uniform',
                                                colorValue=0xff4242, radius=0.15)
    view.add_representation('ball+stick', selection='.B', colorScheme='uniform',
                            colorValue=0x475FD0, radius=0.35)
    view.add_unitcell()
    return view

In [45]:
view = make_animation(2, 7.0)
view

(1001, 3000, 3)
(3000, 3)
1000
(3000, 3)


NGLWidget()